In [1]:
from Gagnepain2012 import Model

In [2]:
lexicon = 'ELP_groupData.csv'
novel1 = 'Novel_Lexicon_1.csv'
novel2 = 'Novel_Lexicon_2.csv'

In [3]:
novel1_Words = [
    x.split(',')[2]
    for x in open (novel1, "r").readlines()[1:]
    ]
novel2_Words = [
    x.split(',')[2]
    for x in open (novel2, "r").readlines()[1:]
    ]

In [4]:
model_Dict = {
    ('Original', 'Non_Frequency'): Model(
        lexicon_File = lexicon,
        additional_Lexicon_File = None,
        use_Frequency= False
        ),
    ('Post_Novel1', 'Non_Frequency'): Model(
        lexicon_File = lexicon,
        additional_Lexicon_File = novel1,
        use_Frequency= False
        ),
    ('Post_Novel2', 'Non_Frequency'): Model(
        lexicon_File = lexicon,
        additional_Lexicon_File = novel2,
        use_Frequency= False
        ),
    ('Original', 'Frequency'): Model(
        lexicon_File = lexicon,
        additional_Lexicon_File = None,
        use_Frequency= True
        ),
    ('Post_Novel1', 'Frequency'): Model(
        lexicon_File = lexicon,
        additional_Lexicon_File = novel1,
        use_Frequency= True
        ),
    ('Post_Novel2', 'Frequency'): Model(
        lexicon_File = lexicon,
        additional_Lexicon_File = novel2,
        use_Frequency= True
        ),
    }

In [5]:
original_Labels, originals, novel1_Labels, novel1s, novel2_Labels, novel2s, *_ = zip(*[
    x.split(',')
    for x in open ('gagnepain_items_revised.csv', 'r').readlines()[1:]
    ])

In [6]:
uniqueness_DIct = {
    'Original': {},
    'Novel1': {},
    'Novel2': {},
    }

for lexicon in originals + novel1s + novel2s:
    candidates = [x for x in model_Dict['Original', 'Non_Frequency'].pronunciation_Dict.values()]
    uniqueness_Point = None
    
    for index in range(1, len(lexicon)):
        candidates = [
            candidate
            for candidate in candidates
            if lexicon[:index] == candidate[:index]
            ]
        if len(candidates) == 1:
            uniqueness_Point = index
            break
    uniqueness_DIct['Original'][lexicon] = uniqueness_Point or len(lexicon) + 1
    
    
for lexicon in originals + novel1s + novel2s:
    candidates = [x for x in model_Dict['Post_Novel1', 'Non_Frequency'].pronunciation_Dict.values()]
    uniqueness_Point = None
    
    for index in range(1, len(lexicon)):
        candidates = [
            candidate
            for candidate in candidates
            if lexicon[:index] == candidate[:index]
            ]
        if len(candidates) == 1:
            uniqueness_Point = index
            break
    uniqueness_DIct['Novel1'][lexicon] = uniqueness_Point or len(lexicon) + 1
    
for lexicon in originals + novel1s + novel2s:
    candidates = [x for x in model_Dict['Post_Novel2', 'Non_Frequency'].pronunciation_Dict.values()]
    uniqueness_Point = None
    
    for index in range(1, len(lexicon)):
        candidates = [
            candidate
            for candidate in candidates
            if lexicon[:index] == candidate[:index]
            ]
        if len(candidates) == 1:
            uniqueness_Point = index
            break
    uniqueness_DIct['Novel2'][lexicon] = uniqueness_Point or len(lexicon) + 1

In [8]:
for frequency in ['Non_Frequency', 'Frequency']:    
    exports = ['', 'PHASE', 'TYPE', 'ORIGINAL', 'ACTUAL', 'UP', 'DP', 'position', 'position_UP', 'position_DP', 'phoneme', 'phon_prob', 'error']
    exports = ['\t'.join(exports)]
        
    for phase, model in [
        (phase, model)
        for (phase, use_Frequency), model in model_Dict.items()
        if use_Frequency == frequency
        ]:
        for dataset in ['Original', 'Novel1', 'Novel2']:
            print(frequency, phase, dataset)
            for dataset_Label, labels, patterns in [
                ('Original', original_Labels, originals),
                ('Novel1', novel1_Labels, novel1s),
                ('Novel2', novel2_Labels, novel2s)
                ]:
                for label, original, pattern in zip(labels, originals, patterns):
                    probabilities, errors, uniqueness_Point = model.Test(pattern)                
                    probabilities = ['{:.5f}'.format(x) for x in probabilities]
                    errors = ['{:.5f}'.format(x) for x in errors]

                    for position, (probability, error) in enumerate(zip(probabilities, errors), 1):
                        line = [len(exports), phase, dataset_Label, original, pattern, uniqueness_DIct['Original'][original], uniqueness_DIct[dataset][pattern], position, '?', '?', pattern[position - 1], probability, error]
                        exports.append('\t'.join([f'{x}' for x in line]))
                        
    open('Gagnepain2012_Results.{}.txt'.format(frequency), 'w').write('\n'.join(exports))

Non_Frequency Original Original
Non_Frequency Original Novel1
Non_Frequency Original Novel2
Non_Frequency Post_Novel1 Original
Non_Frequency Post_Novel1 Novel1
Non_Frequency Post_Novel1 Novel2
Non_Frequency Post_Novel2 Original
Non_Frequency Post_Novel2 Novel1
Non_Frequency Post_Novel2 Novel2
Frequency Original Original
Frequency Original Novel1
Frequency Original Novel2
Frequency Post_Novel1 Original
Frequency Post_Novel1 Novel1
Frequency Post_Novel1 Novel2
Frequency Post_Novel2 Original
Frequency Post_Novel2 Novel1
Frequency Post_Novel2 Novel2
